### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Import & Preprocess

In [97]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)


In [98]:
df = pd.read_csv('merged-essentials.csv')
# df = pd.read_csv('merged-essentials-sep.csv')

df.head()

,user principal name,display name,state,title,department,office,block credential,has onedrive license,onedrive last activity date,has sharepoint license,sharepoint last activity date,has teams license,teams last activity date,report refresh date
0,ar_mubarok@bi.go.id,A Robith Mubarok,KPWDN,Pelaksana,JR.KANTOR PERWAKILAN BI JEMBER,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,False,True,2021-07-13,True,2022-03-20,True,2022-04-04,2022-04-04
1,amirullah@bi.go.id,A. Amirullah Amir,KP,Manajer,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,False,True,2022-03-24,True,2022-03-22,True,2022-04-04,2022-04-04
2,donanto@bi.go.id,A. Donanto H. W.,KP,Direktur,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,False,True,2022-03-30,True,2022-04-04,True,2022-04-04,2022-04-04
3,arasta@bi.go.id,A. Farid Aulia,KP,Deputi Direktur,BINS.INSTITUT BI,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,False,True,2021-10-27,True,2022-03-29,True,2022-03-29,2022-04-04
4,afauzi_b@bi.go.id,A. Fauzi Batubara,KPWDN,Manajer,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,KEPALA UNIT PUR-MDN.SPMI.UNIT LAYANAN & PENGOL...,False,True,2022-03-08,True,2022-03-08,True,2022-04-04,2022-04-04


In [99]:
len(df)

7659

In [100]:
# pcpm baru license nya false? di exclude aja? 
# only take Teams, OneDrive, Share Point = True, True, True => asumsi lisensi 365
df = df.loc[df['has teams license']==True]
len(df)

5509

In [101]:
df = df.loc[df['has onedrive license']==True]
len(df)

5509

In [102]:
df = df.loc[df['has sharepoint license']==True]
len(df)

5509

In [103]:
# to datetime
df['teams last activity date'] = pd.to_datetime(df['teams last activity date'])
df['onedrive last activity date'] = pd.to_datetime(df['onedrive last activity date'])
df['sharepoint last activity date'] = pd.to_datetime(df['sharepoint last activity date'])
df['report refresh date'] = pd.to_datetime(df['report refresh date'])

# calculate days
df['inactive days teams'] = df['report refresh date']-df['teams last activity date']
df['inactive days teams'] = (df['inactive days teams']).dt.days

df['inactive days onedrive'] = df['report refresh date']-df['onedrive last activity date']
df['inactive days onedrive'] = (df['inactive days onedrive']).dt.days

df['inactive days sharepoint'] = df['report refresh date']-df['sharepoint last activity date']
df['inactive days sharepoint'] = (df['inactive days sharepoint']).dt.days



In [104]:
df['active last week onedrive/sharepoint'] = np.where((df['inactive days onedrive']<10) | (df['inactive days sharepoint']<10), 1,0)
df['active last week or'] = np.where((df['inactive days onedrive']<10) | (df['inactive days sharepoint']<10) | (df['inactive days teams']<10), 1,0)
df['active last week all'] = np.where((df['inactive days onedrive']<10) & (df['inactive days sharepoint']<10) & (df['inactive days teams']<10), 1,0)

df['active last week teams'] = np.where(df['inactive days teams']<10, 1,0)
df['active last week onedrive'] = np.where(df['inactive days onedrive']<10, 1,0)
df['active last week sharepoint'] = np.where(df['inactive days sharepoint']<10, 1,0)




In [105]:
# remove wrong data
df = df.loc[(df['state']=='KP')|(df['state']=='KPWDN')|(df['state']=='KPWLN')]

df = df.loc[df['department']!="'-"]

# High Level

In [106]:
df.groupby(by=["state"]).mean()

,block credential,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
state,,,,,,,,,,
KP,0.002799,3.180931,19.955514,23.224159,0.805348,0.936256,0.535448,0.931903,0.758706,0.583644
KPWDN,0.000000,9.575496,46.767123,33.286210,0.568530,0.831272,0.297872,0.788224,0.470559,0.415636
KPWLN,0.000000,0.461538,16.346154,14.346154,0.884615,1.000000,0.500000,1.000000,0.769231,0.615385


In [107]:
# percentage active user in last 1 wk
df.groupby(by=["department"])['active last week all'].mean().sort_values(ascending=False)
# df.groupby(by=["department"]).mean()



department
DMR.DEPARTEMEN MANAJEMEN RISIKO                                 0.826923
DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI                    0.779592
DSTA.DEPARTEMEN STATISTIK                                       0.779528
UKPS.UNIT KHUSUS PEMBANGUNAN SPU DC & BRS                       0.756757
DKMP.DEPARTEMEN KEBIJAKAN MAKROPRUDENSIAL                       0.738636
DPD.DEPARTEMEN PENGELOLAAN DEVISA                               0.725000
DPM.DEPARTEMEN PENGELOLAAN MONETER                              0.720000
SN.KANTOR PERWAKILAN BI SINGAPORE                               0.714286
DUPK.DEPARTEMEN PENGEMBANGAN UMKM & PERLINDUNGAN KONSUMEN       0.700000
DR.DEPARTEMEN REGIONAL                                          0.700000
DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER                     0.693182
DINT.DEPARTEMEN INTERNASIONAL                                   0.681159
DPKL.DEPARTEMEN PENGELOLAAN & KEPATUHAN LAPORAN                 0.655172
DHK.DEPARTEMEN HUKUM                    

# KP

In [108]:
kp = df.loc[df['state']=='KP']
kp.groupby(by=["department"]).mean()

,block credential,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
department,,,,,,,,,,
BINS.INSTITUT BI,0.000000,3.474747,38.595960,12.818182,0.818182,0.969697,0.464646,0.929293,0.555556,0.747475
DAI.DEPARTEMEN AUDIT INTERN,0.000000,0.392405,5.126582,13.202532,0.848101,0.987342,0.607595,0.987342,0.822785,0.632911
DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,0.000000,0.428571,8.102041,14.918367,0.816327,1.000000,0.510204,1.000000,0.775510,0.551020
DHK.DEPARTEMEN HUKUM,0.000000,1.403509,6.736842,14.087719,0.912281,0.982456,0.649123,0.982456,0.842105,0.719298
DINT.DEPARTEMEN INTERNASIONAL,0.000000,0.260870,11.521739,6.971014,0.884058,1.000000,0.681159,1.000000,0.811594,0.753623
DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER,0.000000,0.034091,3.704545,13.556818,0.943182,1.000000,0.693182,1.000000,0.943182,0.693182
DKEU.DEPARTEMEN KEUANGAN,0.009615,0.134615,4.115385,16.057692,0.913462,1.000000,0.634615,1.000000,0.894231,0.653846
DKMP.DEPARTEMEN KEBIJAKAN MAKROPRUDENSIAL,0.000000,0.602273,6.147727,8.238636,0.886364,0.977273,0.738636,0.977273,0.818182,0.806818
DKOM.DEPARTEMEN KOMUNIKASI,0.000000,1.809524,37.395062,45.547619,0.595238,0.952381,0.285714,0.952381,0.547619,0.333333


In [109]:
kp.groupby(by=["department"])['active last week all'].mean().sort_values(ascending=False)

department
DMR.DEPARTEMEN MANAJEMEN RISIKO                                 0.826923
DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI                    0.779592
DSTA.DEPARTEMEN STATISTIK                                       0.779528
UKPS.UNIT KHUSUS PEMBANGUNAN SPU DC & BRS                       0.756757
DKMP.DEPARTEMEN KEBIJAKAN MAKROPRUDENSIAL                       0.738636
DPD.DEPARTEMEN PENGELOLAAN DEVISA                               0.725000
DPM.DEPARTEMEN PENGELOLAAN MONETER                              0.720000
DR.DEPARTEMEN REGIONAL                                          0.700000
DUPK.DEPARTEMEN PENGEMBANGAN UMKM & PERLINDUNGAN KONSUMEN       0.700000
DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER                     0.693182
DINT.DEPARTEMEN INTERNASIONAL                                   0.681159
DPKL.DEPARTEMEN PENGELOLAAN & KEPATUHAN LAPORAN                 0.655172
DHK.DEPARTEMEN HUKUM                                            0.649123
DKEU.DEPARTEMEN KEUANGAN                

In [110]:
kp.groupby(by=["department"])['active last week or'].mean().sort_values(ascending=False)

department
UKTSI.UNIT KHUSUS TRANSFORMASI SISTEM INFORMASI                 1.000000
DKEU.DEPARTEMEN KEUANGAN                                        1.000000
DSTA.DEPARTEMEN STATISTIK                                       1.000000
DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN               1.000000
DPPK.DEPARTEMEN PENGEMBANGAN PASAR KEUANGAN                     1.000000
DMR.DEPARTEMEN MANAJEMEN RISIKO                                 1.000000
DUPK.DEPARTEMEN PENGEMBANGAN UMKM & PERLINDUNGAN KONSUMEN       1.000000
DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER                     1.000000
DINT.DEPARTEMEN INTERNASIONAL                                   1.000000
DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH                      1.000000
DR.DEPARTEMEN REGIONAL                                          1.000000
DPS.DEPARTEMEN PENGADAAN STRATEGIS                              0.988764
DPD.DEPARTEMEN PENGELOLAAN DEVISA                               0.987500
DAI.DEPARTEMEN AUDIT INTERN             

# KPWDN

In [80]:
kpwdn = df.loc[df['state']=='KPWDN']
kpwdn.groupby(by=["department"]).mean()

,block credential,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
department,,,,,,,,,,
AB.KANTOR PERWAKILAN BI PROVINSI MALUKU,0.000000,10.083333,19.828571,22.833333,0.861111,0.944444,0.583333,0.944444,0.694444,0.750000
BD.KANTOR PERWAKILAN BI PROVINSI JAWA BARAT,0.000000,26.195122,57.202532,20.707317,0.792683,0.865854,0.341463,0.768293,0.426829,0.731707
BDL.KANTOR PERWAKILAN BI PROVINSI LAMPUNG,0.000000,5.864407,66.779661,11.576271,0.694915,0.915254,0.220339,0.881356,0.220339,0.694915
BJM.KANTOR PERWAKILAN BI PROVINSI KALIMANTAN SELATAN,0.000000,28.652778,51.338235,14.125000,0.888889,0.888889,0.541667,0.777778,0.638889,0.861111
BN.KANTOR PERWAKILAN BI PROVINSI BENGKULU,0.000000,44.055556,107.756757,53.947368,0.710526,0.736842,0.342105,0.657895,0.447368,0.631579
BNA.KANTOR PERWAKILAN BI PROVINSI ACEH,0.000000,24.473684,63.702703,13.230769,0.743590,0.820513,0.435897,0.743590,0.487179,0.717949
BPP.KANTOR PERWAKILAN BI BALIKPAPAN,0.000000,7.375000,43.921053,2.275000,0.950000,0.975000,0.500000,0.925000,0.500000,0.950000
BTM.KANTOR PERWAKILAN BI PROVINSI KEPULAUAN RIAU,0.000000,15.510204,40.489796,27.530612,0.816327,0.857143,0.653061,0.816327,0.653061,0.816327
CN.KANTOR PERWAKILAN BI CIREBON,0.000000,34.622222,46.000000,30.829787,0.744681,0.829787,0.297872,0.702128,0.382979,0.659574


In [81]:
kpwdn.groupby(by=["department"])['active last week all'].mean().sort_values(ascending=False)

department
MJU.KANTOR PERWAKILAN BI PROVINSI SULAWESI BARAT               0.677419
BTM.KANTOR PERWAKILAN BI PROVINSI KEPULAUAN RIAU               0.653061
JKT.KANTOR PERWAKILAN BI PROVINSI DKI JAKARTA                  0.617647
PKP.KANTOR PERWAKILAN BI PROVINSI KEPULAUAN BANGKA BELITUNG    0.606061
AB.KANTOR PERWAKILAN BI PROVINSI MALUKU                        0.583333
GTO.KANTOR PERWAKILAN BI PROVINSI GORONTALO                    0.583333
BJM.KANTOR PERWAKILAN BI PROVINSI KALIMANTAN SELATAN           0.541667
TGL.KANTOR PERWAKILAN BI TEGAL                                 0.526316
JAP.KANTOR PERWAKILAN BI PROVINSI PAPUA                        0.521739
PLK.KANTOR PERWAKILAN BI PROVINSI KALIMANTAN TENGAH            0.520833
TRK.KANTOR PERWAKILAN BI PROVINSI KALIMANTAN UTARA             0.515152
KPA.KANTOR PERWAKILAN BI PROVINSI NUSA TENGGARA TIMUR          0.511628
PTK.KANTOR PERWAKILAN BI PROVINSI KALIMANTAN BARAT             0.510204
BPP.KANTOR PERWAKILAN BI BALIKPAPAN                  

# KPWLN

In [82]:
kpwln = df.loc[df['state']=='KPWLN']
kpwln.groupby(by=["department"]).mean()

,block credential,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
department,,,,,,,,,,
BJ.KANTOR PERWAKILAN BI BEIJING,0.0,0.500000,41.750000,7.750000,0.75,1.0,0.500000,1.0,0.500000,0.750000
LND.KANTOR PERWAKILAN BI LONDON,0.0,0.142857,15.571429,3.285714,1.00,1.0,0.714286,1.0,0.857143,0.857143
NY.KANTOR PERWAKILAN BI NEW YORK,0.0,0.285714,2.285714,0.857143,1.00,1.0,1.000000,1.0,1.000000,1.000000
SN.KANTOR PERWAKILAN BI SINGAPORE,0.0,0.125000,10.750000,0.500000,1.00,1.0,0.750000,1.0,0.750000,1.000000
TKY.KANTOR PERWAKILAN BI TOKYO,0.0,1.500000,7.000000,5.500000,1.00,1.0,0.500000,1.0,0.750000,0.750000


# Check


In [83]:
df[df['department']=='DKOM.DEPARTEMEN KOMUNIKASI']


,user principal name,display name,state,title,department,office,block credential,has onedrive license,onedrive last activity date,has sharepoint license,sharepoint last activity date,has teams license,teams last activity date,report refresh date,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
17,aan_sm@bi.go.id,Aan Sari Mayani,KP,Asisten Direktur,DKOM.DEPARTEMEN KOMUNIKASI,ANALIS SENIOR-DKOM.PPKO.FUNGSI PERENCANAAN & S...,False,True,2022-09-07,True,2022-09-07,True,2022-09-08,2022-09-08,0.0,1.0,1.0,1,1,1,1,1,1
67,abraham_wn@bi.go.id,Abraham Wahyu Nugroho,KP,Manajer,DKOM.DEPARTEMEN KOMUNIKASI,ANALIS-DKOM.RLP2.FUNGSI KEMENTERIAN/LPNK BIDAN...,False,True,2022-09-06,True,2022-09-06,True,2022-09-07,2022-09-08,1.0,2.0,2.0,1,1,1,1,1,1
119,ade_ia@bi.go.id,Ade Imani Arsyad,KP,Pelaksana,DKOM.DEPARTEMEN KOMUNIKASI,PELAKSANA-DKOM.PMU.FUNGSI OPERASIONAL MUSEUM BI,False,True,2022-09-02,True,2022-09-02,True,2022-09-08,2022-09-08,0.0,6.0,6.0,1,1,1,1,1,1
228,adrie_h.i@bi.go.id,Adrie Handria (PKWT),KP,Asisten Manajer,DKOM.DEPARTEMEN KOMUNIKASI,PEGAWAI PKWT-DKOM.DEPARTEMEN KOMUNIKASI,False,True,2022-09-06,True,2022-09-02,True,2022-09-08,2022-09-08,0.0,2.0,6.0,1,1,1,1,1,1
242,afif_ap@bi.go.id,Afif Anggoro Prasetyo,KP,Pelaksana,DKOM.DEPARTEMEN KOMUNIKASI,ADMINISTRATOR-DKOM.LIPK.TIM PUBLIKASI EKSTERNAL,False,True,2022-08-30,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,9.0,0.0,1,1,1,1,1,1
289,a_wijanarko@bi.go.id,Agung Wijanarko,KP,Pelaksana,DKOM.DEPARTEMEN KOMUNIKASI,PELAKSANA-DKOM.PMU.FUNGSI KOLEKSI & TATA PAMER...,False,True,2022-09-06,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,2.0,0.0,1,1,1,1,1,1
301,agus_llb@bi.go.id,Agus Langlang Buana,KP,Manajer,DKOM.DEPARTEMEN KOMUNIKASI,ANALIS-DKOM.RLP1.FUNGSI RELASI PARLEMEN,False,True,2022-05-09,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,122.0,0.0,1,1,0,1,0,1
335,agustiara_purba@bi.go.id,Agustiara Purba,KP,Asisten Manajer,DKOM.DEPARTEMEN KOMUNIKASI,ANALIS YUNIOR-DKOM.LIPK.TIM PUBLIKASI EKSTERNAL,False,True,2022-09-08,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,0.0,0.0,1,1,1,1,1,1
859,anyrama@bi.go.id,Any Ramadhaningsih,KP,Manajer,DKOM.DEPARTEMEN KOMUNIKASI,ANALIS-DKOM.LIPK.TIM PUBLIKASI EKSTERNAL,False,True,2022-08-29,True,2022-09-04,True,2022-09-07,2022-09-08,1.0,10.0,4.0,1,1,0,1,0,1
938,arief_dh@bi.go.id,Arief Dwi Handoko,KP,Pelaksana,DKOM.DEPARTEMEN KOMUNIKASI,ASISTEN ADMINISTRATOR-DKOM.TMI.FUNGSI LOGISTIK...,False,True,2022-08-11,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,28.0,0.0,1,1,0,1,0,1


In [84]:
# more than 1 month inactive on teams, resign??
df.loc[df['teams last activity date']<pd.to_datetime('2022-03-01')].head()

,user principal name,display name,state,title,department,office,block credential,has onedrive license,onedrive last activity date,has sharepoint license,sharepoint last activity date,has teams license,teams last activity date,report refresh date,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
15,athinih_sufiyati.i@bi.go.id,A’thinih Sufiyati (PKWT),KPWDN,Staf,JKT.KANTOR PERWAKILAN BI PROVINSI DKI JAKARTA,PEGAWAI PKWT-JKT.KANTOR PERWAKILAN BI PROVINSI...,False,True,2022-09-08,True,2022-09-06,True,2021-08-09,2022-09-08,395.0,0.0,2.0,1,1,0,0,1,1
19,aang_p@bi.go.id,Aang Parade,KPWDN,Pelaksana,BN.KANTOR PERWAKILAN BI PROVINSI BENGKULU,PENGAWAS PAM YUNIOR-BN.SPUR.FUNGSI LOGISTIK SD...,False,True,2022-03-24,True,2022-03-24,True,2021-12-10,2022-09-08,272.0,168.0,168.0,0,0,0,0,0,0
41,a_rasyid@bi.go.id,Abdul Rasyid,KPWDN,Asisten,JB.KANTOR PERWAKILAN BI PROVINSI JAMBI,ASISTEN PENGAWAS PAM-JB.SPUR.FUNGSI LOGISTIK S...,False,True,2022-08-13,True,2022-09-08,True,2021-12-16,2022-09-08,266.0,26.0,0.0,1,1,0,0,0,1
52,abdurahman_zm.i@bi.go.id,Abdurahman Zaki Mustofa (PKWT),KP,Staf,DPKL.DEPARTEMEN PENGELOLAAN & KEPATUHAN LAPORAN,PEGAWAI PKWT-DPKL.DEPARTEMEN PENGELOLAAN & KEP...,True,True,2022-01-28,True,2021-12-02,True,2022-02-04,2022-09-08,216.0,223.0,280.0,0,0,0,0,0,0
77,achmad_01@bi.go.id,Achmad,KPWDN,Staf,SB.KANTOR PERWAKILAN BI PROVINSI JAWA TIMUR,ASISTEN PENYELIA.-SB.TMI.FUNGSI LOGISTIK SDM S...,False,True,NaT,True,2021-09-16,True,2021-11-17,2022-09-08,295.0,NaN,357.0,0,0,0,0,0,0


In [85]:
# more than 1 month inactive on teams, resign??
print(len(df.loc[df['teams last activity date']<pd.to_datetime('2022-03-01')])) 

134


In [86]:
a = df.groupby(by=["state"]).mean()
b = kp.groupby(by=["department"]).mean()
c = kpwdn.groupby(by=["department"]).mean()
d = kpwln.groupby(by=["department"]).mean()


with pd.ExcelWriter('user-active-percentage-sep.xlsx') as writer:  
    a.to_excel(writer, sheet_name='kp vs kpw vs kpwln')
    b.to_excel(writer, sheet_name='KP')
    c.to_excel(writer, sheet_name='KPWDN')
    d.to_excel(writer, sheet_name='KPWLN')

# a.to_csv('active-percentage-kp-kpw.csv')
# b.to_csv('active-percentage-kp.csv')
# c.to_csv('active-percentage-kpwdn.csv')
# d.to_csv('active-percentage-kpwln.csv')

In [88]:
df.head()

,user principal name,display name,state,title,department,office,block credential,has onedrive license,onedrive last activity date,has sharepoint license,sharepoint last activity date,has teams license,teams last activity date,report refresh date,inactive days teams,inactive days onedrive,inactive days sharepoint,active last week onedrive/sharepoint,active last week or,active last week all,active last week teams,active last week onedrive,active last week sharepoint
0,ar_mubarok@bi.go.id,A Robith Mubarok,KPWDN,Pelaksana,JR.KANTOR PERWAKILAN BI JEMBER,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,False,True,2022-07-16,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,54.0,0.0,1,1,0,1,0,1
1,amirullah@bi.go.id,A. Amirullah Amir,KP,Manajer,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,False,True,2022-09-05,True,2022-09-07,True,2022-09-08,2022-09-08,0.0,3.0,1.0,1,1,1,1,1,1
2,donanto@bi.go.id,A. Donanto H. W.,KP,Direktur,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,KEPALA GRUP-DPSP.GRUP ELEKTRONIFIKASI & STANDA...,False,True,2022-08-29,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,10.0,0.0,1,1,0,1,0,1
3,arasta@bi.go.id,A. Farid Aulia,KP,Deputi Direktur,BINS.INSTITUT BI,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,False,True,2021-10-27,True,2022-08-24,True,2022-08-01,2022-09-08,38.0,316.0,15.0,0,0,0,0,0,0
4,afauzi_b@bi.go.id,A. Fauzi Batubara,KPWDN,Manajer,BD.KANTOR PERWAKILAN BI PROVINSI JAWA BARAT,KEPALA UNIT PUR-BD.SPMI.UNIT LAYANAN & PENGOLA...,False,True,2022-09-06,True,2022-09-08,True,2022-09-08,2022-09-08,0.0,2.0,0.0,1,1,1,1,1,1


In [95]:
# September
df['active last week all'].mean()

0.6050832602979842

In [111]:
# April
df['active last week all'].mean()

0.44404332129963897